In [1]:
import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.table import QTable
import matplotlib.pyplot as plt
import pdb
%matplotlib inline

In [2]:
const.G

<<class 'astropy.constants.codata2014.CODATA2014'> name='Gravitational constant' value=6.67408e-11 uncertainty=3.1e-15 unit='m3 / (kg s2)' reference='CODATA 2014'>

In [3]:
def calc_dmdt(eta, F_xuv, R_xuv, M_p):
    dmdt = (eta * np.pi * F_xuv * R_xuv**3)/(const.G * M_p)
    return dmdt

def calc_M(dmdt,dt):
    M = dmdt * dt
    return M

def calc_H(Rg, T, gs):
    H = Rg * T / gs
    return H

def calc_gs(m,r):
    g = const.G * m / r**2
    return g

def calc_ps(gs, Menv, dM, Rs):
    ps = gs * (Menv - dM) / (4 * np.pi * Rs**2)
    return ps

def calc_Rxuv(Rs, H, Pxuv, Ps):
    Rxuv = Rs**2 / (H * np.log(Pxuv/Ps) + Rs)
    return Rxuv

In [8]:
# reproduce figure 3
# 
alpha = 0.03
Mp = 2
Menv = alpha * Mp
Mcore = Mp - Menv

Mp = Mp * const.M_earth
Menv = Menv * const.M_earth
Mcore = Mcore * const.M_earth


#Rs = (Mcore.value/1.3)**(1/3.7) * u.m
Rs = 1.3 * (Mcore/u.kg) ** 0.27 *u.m # I dont think this is right? 

T = 880 * u.K
a = 0.1 * u.AU
Mstar = 1 * const.M_sun
Fxuv = 55 * u.W / (u.m**2)
eta = 0.1
Pxuv = 5 * u.Pa
Rg = 4157 * u.J / (u.kg * u.K) # stated as 4157 in paper, actually used 4124
tao = 1e8 * u.yr

Mp, Menv, Mcore, Rs




(<Quantity 1.1944729460839545e+25 kg>,
 <Quantity 3.5834188382518635e+23 kg>,
 <Quantity 1.1586387577014358e+25 kg>,
 <Quantity 7606928.362559093 m>)

In [9]:
t = 0 * u.yr
dt = 10000 * u.yr
dM0 = 0 * u.kg

# constants
g = calc_gs(Mcore, Rs)
H = calc_H(Rg, T, g)

# intial conditions
P0 = calc_ps(g, Menv, dM0, Rs)
P0 = P0.to(u.Pa)
Rxuv0 = calc_Rxuv(Rs, H, Pxuv, P0)
Rxuv0 = Rxuv0.decompose()
dmdt0 = calc_dmdt(eta, Fxuv, Rxuv0, Mcore)
dmdt0 = dmdt0.decompose()

data = QTable()
# defining some initial conditions
data['time'] = [0]*u.yr
data['Mp'] = [Mp.decompose().value] * u.kg
data['alpha'] = [alpha]
data['Menv'] = [Menv.decompose().value] * u.kg
data['P_surf'] = [P0.decompose().value] * u.Pa
data['R_xuv'] = [Rxuv0.decompose().value] * u.m
data['dMdt'] = [dmdt0.decompose().value] * u.kg/u.s
data['dM'] = [(dmdt0 * dt).decompose().value] * u.kg
data

time,Mp,alpha,Menv,P_surf,R_xuv,dMdt,dM
yr,kg,,kg,Pa,m,kg / s,kg
float64,float64,float64,float64,float64,float64,float64,float64
0.0,1.1944729460839545e+25,0.03,3.5834188382518635e+23,6585516817.569603,31132233.751170516,674223599.6354682,2.1276878667856252e+20


In [13]:
g, Rs, H, P0, Rxuv0, Menv/const.M_earth, 3.0390977564e+07/const.R_earth

(<Quantity 13.36351813679245 m / s2>,
 <Quantity 7606928.362559093 m>,
 <Quantity 273742.28571803635 J s2 / (kg m)>,
 <Quantity 6585516817.569603 Pa>,
 <Quantity 31132233.751170516 m>,
 <Quantity 0.06>,
 <Quantity 4.780861056506418 1 / m>)

In [20]:
test = 4.263493e-05 * u.au
#test.to(u.earthRad)
new_test = 3.0390977564e+07 * u.m
new_test.to(u.earthRad)

<Quantity 4.780861056506418 earthRad>

In [21]:
m = 1.1944372e+25 * u.kg
m.to(u.earthMass)

<Quantity 1.9999401475201732 earthMass>

In [ ]:


while data['time'][-1] < tao:
    #pdb.set_trace()
    t = data['time'][-1]
    t_new = t + dt
    
    Menv_new = data['Menv'][-1] - data['dM'][-1]
    Mp_new = Mcore+Menv_new
    alpha_new = Menv_new/Mp_new
    
    Ps_new = calc_ps(g, Menv_new, data['dM'][-1], Rs).decompose()
    Rxuv_new = calc_Rxuv(Rs, H, Pxuv, Ps_new).decompose()
    dmdt_new = calc_dmdt(eta, Fxuv, Rxuv_new, Mcore).decompose()
    
    dm_new = (dmdt_new * dt).decompose()
    data.add_row((t_new, Mp_new, alpha_new, Menv_new, Ps_new, Rxuv_new, dmdt_new, dm_new))
    if Ps_new <= 0:
        break
    
    

In [ ]:
time = data['time']
Menv = data['Menv']
Mp = data['Mp']
Psurf = data['P_surf']
Rxuv = data['R_xuv']
dmdt = data['dMdt']
dm = data['dM']


fig,ax = plt.subplots(3,1)
fig.set_size_inches(7,7)
fig.subplots_adjust(bottom=0.3)

ax[0].plot(time, Menv/Menv[0], color='blue')
ax[0].set(ylabel='Atmospheric Mass\nFraction', xlim=(0,30e6), ylim=(0,1))

ax[1].plot(time, Rxuv/Rxuv[0], color='blue')
ax[1].set(ylabel='Rxuv')

ax[2].plot(time, Mp/Mp[0], color='blue')
ax[2].set(ylabel='Mp', xlabel='Time (yr)')


    

fig.savefig('atmesc.pdf')

In [ ]:
#data[2840:2850]
data[:5]

In [ ]:
lehm_final_time = 28644016 * u.yr
diego_final_time = data['time'][-1]
(diego_final_time - lehm_final_time)/(lehm_final_time)

In [ ]:
diego_final_time

In [ ]:
lehm_final_time

In [ ]:
1/3.7

In [ ]:
c_data = QTable.read('lehmer/structs/pointers/output.csv', format='csv')

In [ ]:
c_data = QTable.read('lehmer/structs/pointers/output.csv', format='csv')
c_time = c_data['time']
c_Menv = c_data['Menv']
c_Rxuv = c_data['Rxuv']
c_Mp = c_data['Mp']

fig,ax = plt.subplots(3,1)
fig.set_size_inches(7,7)
fig.subplots_adjust(bottom=0.3)

ax[0].plot(c_time, c_Menv/c_Menv[0], color='blue')
ax[0].set(ylabel='Atmospheric Mass\nFraction', xlim=(0,30e6), ylim=(0,1))

ax[1].plot(c_time, c_Rxuv/c_Rxuv[0], color='blue')
ax[1].set(ylabel='Rxuv')

ax[2].plot(c_time, c_Mp/c_Mp[0], color='blue')
ax[2].set(ylabel='Mp', xlabel='Time (yr)')

In [ ]:
c_time[-1]

In [ ]:
c_Menv/c_Menv[0]